## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [6]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

%matplotlib inline

In [7]:
effect_size = sms.proportion_effectsize(0.20, 0.19)    # Розрахунок розміру ефекту на основі наших очікуваних показників

In [8]:
effect_size

np.float64(0.025241594409087353)

In [9]:
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
    )                                                  # Розрахунок необхідного розміру вибірки

required_n = ceil(required_n)                          # Округлення до наступного цілого числа

print(required_n)

24638


✅ Висновок:
Щоб виявити зростання утримання з 19% до 20% з потужністю 80% і α = 0.05, нам потрібно приблизно 49 тисяч користувачів (по 24 638 спостережень для кожної групи). 

2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [15]:
df = pd.read_csv('cookie_cats.csv')

df.head()

# Обчислення середнього retention_7 по версіях гри
retention_by_version = df.groupby("version")["retention_7"].mean()
print("Середнє утримання на 7 день по версіях:")
print(retention_by_version)

# Формулювання гіпотези
# Середнє утримання на 7 день однакове для обох версій (gate_30 = gate_40)
# Середнє утримання на 7 день відрізняється між версіями (gate_30 ≠ gate_40)

Середнє утримання на 7 день по версіях:
version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


✅*Гіпотеза*: Утримання на 7 день трохи краще у версії gate_30 (контрольної групи). Але потрібно статистично перевірити, чи ця різниця є значущою (наприклад, через z-тест для двох пропорцій).

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [16]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
import numpy as np

# Крок 1. Підрахунок кількості успіхів (retention_7 = 1) та загальної кількості для кожної версії
counts = df.groupby("version")["retention_7"].sum()
nobs = df.groupby("version")["retention_7"].count()

# Приведемо до масивів у правильному порядку (gate_30, gate_40)
successes = np.array([counts["gate_30"], counts["gate_40"]])
observations = np.array([nobs["gate_30"], nobs["gate_40"]])

# Крок 2. Виконання Z-тесту для двох пропорцій
z_stat, p_val = proportions_ztest(successes, observations)

# Крок 3. Обчислення довірчих інтервалів для обох груп
confint_control = proportion_confint(count=successes[0], nobs=observations[0], alpha=0.05, method='normal')
confint_treatment = proportion_confint(count=successes[1], nobs=observations[1], alpha=0.05, method='normal')

# Крок 4. Вивід результатів
print(f"\nz statistic: {z_stat:.4f}")
print(f"p-value: {p_val:.4f}")
print(f"Довірчий інтервал 95% для групи control (gate_30): [{confint_control[0]:.4f}, {confint_control[1]:.4f}]")
print(f"Довірчий інтервал 95% для групи treatment (gate_40): [{confint_treatment[0]:.4f}, {confint_treatment[1]:.4f}]")



z statistic: 3.1644
p-value: 0.0016
Довірчий інтервал 95% для групи control (gate_30): [0.1866, 0.1938]
Довірчий інтервал 95% для групи treatment (gate_40): [0.1785, 0.1855]


✅ Висновок:
1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?
Так, різниця є статистично значущою, оскільки p-value = 0.0016 < 0.05, і Z-статистика 3.1644 суттєво перевищує критичне значення для α = 0.05 (≈ 1.96). Це означає, що ми відхиляємо H₀ і приймаємо H₁: утримання на 7-й день відрізняється між групами.

2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?
Ні, довірчі інтервали не перетинаються:

gate_30: [0.1866, 0.1938]

gate_40: [0.1785, 0.1855]

Відсутність перетину довірчих інтервалів додатково підтверджує, що різниця між групами не випадкова і статистично значуща. Це означає, що переміщення воріт зменшило утримання користувачів (бо показник у gate_40 нижчий).

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


*Формулювання гіпотез*

H₀ (нульова гіпотеза):
Версія гри (gate_30 або gate_40) не впливає на утримання гравців на 7-й день (retention_7).
→ Немає залежності між версією та retention_7.

Hа (альтернативна гіпотеза):
Версія гри впливає на утримання гравців на 7-й день.
→ Є залежність між версією та retention_7.

Тест: χ²-тест незалежності (Chi-Square Test of Independence).
Рівень значущості: α = 0.05.

In [21]:
# Створення таблиці спряженості (contingency table)
contingency_table = pd.crosstab(df['version'], df['retention_7'])
print("Таблиця спряженості:")
print(contingency_table)

# Виконання χ²-тесту
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Виведення результатів
print(f"\nChi-square statistic: {chi2:.4f}")
print(f"Degrees of freedom: {dof}")
print(f"P-value: {p:.4f}")

# Висновок
alpha = 0.05
if p < alpha:
    print("Відхиляємо H₀: Є статистично значуща залежність між версією гри та утриманням на 7-й день.")
else:
    print("Не відхиляємо H₀: Немає статистично значущої залежності між версією гри та утриманням на 7-й день.")


Таблиця спряженості:
retention_7  False  True 
version                  
gate_30      36198   8502
gate_40      37210   8279

Chi-square statistic: 9.9591
Degrees of freedom: 1
P-value: 0.0016
Відхиляємо H₀: Є статистично значуща залежність між версією гри та утриманням на 7-й день.


✅ Висновок: Значення p-value = 0.0016, що значно менше рівня значущості α = 0.05.

Це означає, що ми відхиляємо нульову гіпотезу (H₀) та робимо висновок:
між версією гри (gate_30 або gate_40) та утриманням гравців на 7-й день існує статистично значуща залежність.

Тобто, переміщення воріт з рівня 30 на рівень 40 впливає на ймовірність того, що гравець повернеться у гру на 7-й день.

З попередніх обчислень видно, що у версії gate_30 (ворота на рівні 30) утримання трохи вище, ніж у gate_40. Це може означати, що розміщення воріт раніше (на рівні 30) краще утримує гравців у довгостроковій перспективі, ніж розміщення на рівні 40.